<a href="https://colab.research.google.com/github/dcdlima/COS738/blob/main/COS738_Ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -O dataset.zip https://github.com/LINE-PESC/Dados-Exercicios-Texto/raw/main/Base%20para%20usar%20no%20trabalho%20CysticFibrosis2-20240401.zip
!unzip dataset.zip
!rm -rf dataset.zip
!rm -rf sample_data

!pip install nltk

--2025-07-24 11:25:42--  https://github.com/LINE-PESC/Dados-Exercicios-Texto/raw/main/Base%20para%20usar%20no%20trabalho%20CysticFibrosis2-20240401.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/LINE-PESC/Dados-Exercicios-Texto/main/Base%20para%20usar%20no%20trabalho%20CysticFibrosis2-20240401.zip [following]
--2025-07-24 11:25:42--  https://raw.githubusercontent.com/LINE-PESC/Dados-Exercicios-Texto/main/Base%20para%20usar%20no%20trabalho%20CysticFibrosis2-20240401.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1548260 (1.5M) [application/zip]
Saving to: ‘dataset.zip’

dataset.zip     

In [7]:
from pathlib import Path

configs_dir = Path('configs')
configs_dir.mkdir(exist_ok=True)

with open(configs_dir/'PC.CFG', 'w') as conf:
    conf.writelines('LEIA=data/cfquery.xml\n')
    conf.write('CONSULTAS=consultas.csv\n')
    conf.write('ESPERADOS=esperados.csv')

with open(configs_dir/'GLI.CFG', 'w') as conf:
    conf.write('LEIA=data/cf74.xml\n')
    conf.write('LEIA=data/cf75.xml\n')
    conf.write('LEIA=data/cf76.xml\n')
    conf.write('LEIA=data/cf77.xml\n')
    conf.write('LEIA=data/cf78.xml\n')
    conf.write('LEIA=data/cf79.xml\n')
    conf.write('ESCREVA=li.csv')

with open(configs_dir/'INDEX.CFG', 'w') as conf:
    conf.write('LEIA=li.csv\n')
    conf.write('ESCREVA=modelo_vetorial.json')

with open(configs_dir/'BUSCA.CFG', 'w') as conf:
    conf.write('MODELO=modelo_vetorial.json\n')
    conf.write('CONSULTAS=consultas.csv\n')
    conf.write('RESULTADOS=RESULTADOS.csv')

In [8]:
import xml.etree.ElementTree as ET
import csv
import unicodedata
import string
import logging
import time

# Função para normalizar texto
def normalizar_texto(texto):
    texto = texto.upper()
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')
    texto = texto.translate(str.maketrans('', '', string.punctuation + ';'))
    return texto.strip()

# Iniciar log
logging.basicConfig(filename='processador_consultas.log', level=logging.INFO, format='%(asctime)s - %(message)s')
inicio_total = time.time()
logging.info("Início do Processador de Consultas")

# Ler arquivo de configuração PC.CFG
cfg_file = 'configs/PC.CFG'
with open(cfg_file, 'r', encoding='utf-8') as f:
    linhas_cfg = [linha.strip() for linha in f.readlines()]

arquivo_xml = linhas_cfg[0].split('=')[1]
arquivo_consultas = linhas_cfg[1].split('=')[1]
arquivo_esperados = linhas_cfg[2].split('=')[1]
logging.info(f"Arquivo de configuração lido: {cfg_file}")
logging.info(f"Arquivo XML: {arquivo_xml}, Consultas: {arquivo_consultas}, Esperados: {arquivo_esperados}")

# Ler e processar o arquivo XML
tree = ET.parse(arquivo_xml)
root = tree.getroot()
logging.info("Arquivo XML carregado")

# Gerar arquivo de consultas normalizadas
with open(arquivo_consultas, 'w', newline='', encoding='utf-8') as f_consultas:
    writer_consultas = csv.writer(f_consultas, delimiter=';')
    writer_consultas.writerow(['QueryNumber', 'QueryText'])

    for query in root.findall('QUERY'):
        query_number = query.find('QueryNumber').text.strip()
        query_text = query.find('QueryText').text.strip()
        query_text_normalizado = normalizar_texto(query_text)
        writer_consultas.writerow([query_number, query_text_normalizado])

logging.info("Arquivo de consultas normalizadas gerado")

# Gerar arquivo de resultados esperados
with open(arquivo_esperados, 'w', newline='', encoding='utf-8') as f_esperados:
    writer_esperados = csv.writer(f_esperados, delimiter=';')
    writer_esperados.writerow(['QueryNumber', 'DocNumber', 'DocVotes'])

    for query in root.findall('QUERY'):
        query_number = query.find('QueryNumber').text.strip()
        for item in query.find('Records').findall('Item'):
            score = int(item.attrib.get('Score', '0'))
            if score > 0:
                doc_number = item.text.strip()
                writer_esperados.writerow([query_number, doc_number, score])

logging.info("Arquivo de resultados esperados gerado")

fim_total = time.time()
logging.info(f"Processamento concluído em {fim_total - inicio_total:.2f} segundos")
print("Processador de Consultas executado com sucesso.")

Processador de Consultas executado com sucesso.
